In [1]:
from dotenv import load_dotenv
load_dotenv("/Users/koundinya/Documents/prathyusha/projects/expense-management-assistant/env/.env")
import sys
sys.path.append("/Users/koundinya/Documents/prathyusha/projects/expense-management-assistant")
from agent import *
import json

In [2]:
image_path = "/Users/koundinya/Documents/prathyusha/projects/bills/electricity_bill.jpg"
agent = ExpenseAgent()

/Users/koundinya/Documents/prathyusha/projects/expense-management-assistant/agent.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.query_llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")
/Users/koundinya/Documents/prathyusha/projects/expense-management-assistant/agent.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer 

In [3]:
raw_expenses = agent.scan_bill(image_path)
print("Raw Expenses:", raw_expenses)

Raw Expenses: ParsedChatCompletion[ExpenseReport](id='chatcmpl-AgJMmkexDnEs5bMJU6kVv7OoL9UFl', choices=[ParsedChoice[ExpenseReport](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[ExpenseReport](content='{"expenses":[{"expense_id":1,"date":"2009-06-26","description":"Electricity Bill","amount":6440.00}],"total_spent":6440.00}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=ExpenseReport(expenses=[Expense(expense_id=1, date='2009-06-26', description='Electricity Bill', amount=6440.0)], total_spent=6440.0)))], created=1734647584, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_39a40c96a0', usage=CompletionUsage(completion_tokens=41, prompt_tokens=14375, total_tokens=14416, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio

In [4]:
expense_report = json.loads(raw_expenses.choices[0].message.parsed.model_dump_json())

In [5]:
expense_report = raw_expenses.choices[0].message.parsed
expense_report


ExpenseReport(expenses=[Expense(expense_id=1, date='2009-06-26', description='Electricity Bill', amount=6440.0)], total_spent=6440.0)

In [6]:
categorized_expense_report = agent.categorize_expenses(expense_report)
categorized_expense_report = categorized_expense_report.choices[0].message.parsed

In [7]:
categorized_expense_report

CategorizedExpenseReport(categorized_expenses=[CategorizedExpense(expense_id=1, description='Electricity Bill', category='Utilities', amount=6440.0, date='2009-06-26')])

In [8]:
category_expense_report = agent.calculate_expenses(categorized_expense_report)

In [9]:
category_expense_report

{'Utilities': 6440.0}

In [10]:
expense_report

ExpenseReport(expenses=[Expense(expense_id=1, date='2009-06-26', description='Electricity Bill', amount=6440.0)], total_spent=6440.0)

In [11]:
expense_report2 = expense_report.model_copy()

In [12]:
merged_expenses = expense_report.expenses + expense_report2.expenses

total_spent = expense_report.total_spent + expense_report2.total_spent
new_er = ExpenseReport(expenses=merged_expenses, total_spent=total_spent)

In [13]:
new_er

ExpenseReport(expenses=[Expense(expense_id=1, date='2009-06-26', description='Electricity Bill', amount=6440.0), Expense(expense_id=1, date='2009-06-26', description='Electricity Bill', amount=6440.0)], total_spent=12880.0)

In [14]:
categorized_expense_report2 = agent.categorize_expenses(new_er)
categorized_expense_report2 = categorized_expense_report2.choices[0].message.parsed

In [15]:
categorized_expense_report2

CategorizedExpenseReport(categorized_expenses=[CategorizedExpense(expense_id=1, description='Electricity Bill', category='Utilities', amount=6440.0, date='2009-06-26'), CategorizedExpense(expense_id=1, description='Electricity Bill', category='Utilities', amount=6440.0, date='2009-06-26')])

In [16]:
category_expense_report = agent.calculate_expenses(categorized_expense_report2)

In [17]:
category_expense_report

{'Utilities': 12880.0}

In [18]:
query = "What is the total spent on Utilities?"
context = {"expenses": category_expense_report}
response = agent.answer_query(query, context)



> Entering new AgentExecutor chain...
To answer the query about the total spent on Utilities, I need to extract the relevant information from the provided context. 

Action: None
Action Input: None
Observation: None is not a valid tool, try one of [categorize_expenses, calculate_category_expenses].
Thought:To find the total spent on Utilities, I will directly use the data provided in the context.

Action: None
Action Input: None
Observation: None is not a valid tool, try one of [categorize_expenses, calculate_category_expenses].
Thought:I have the information regarding the expenses on Utilities directly from the context provided. 

Action: None
Action Input: None
Observation: None is not a valid tool, try one of [categorize_expenses, calculate_category_expenses].
Thought:I have the total expenses for Utilities directly in the context. The total spent on Utilities is provided as 12880.0.

Final Answer: 12880.0

> Finished chain.


In [24]:
response['output']

'12880.0'